In [1]:
"""
Jason Stranne
"""
import numpy as np
import os
import sys
import gc
from RP_Downstream_Trainer import DownstreamNet, Downstream_Dataset, print_class_counts, num_correct, reduce_dataset_size
from RP_Downstream_Trainer import smallest_class_len, restrict_training_size_per_class
from RP_train_all_at_once import train_end_to_end_RP_combined
sys.path.insert(0, '..')
from Stager_net_pratice import StagerNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
from Custom_data_loader import Custom_RP_Dataset


In [2]:
root = os.path.join("..","training", "")

## Setting up RP
datasets_list=[]
print('Loading Data')
f=open(os.path.join("..","training_names.txt"),'r')
lines = f.readlines()
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    datasets_list.append(Custom_RP_Dataset(path=data_file, total_points=2000, tpos=120, tneg=300, windowSize=30, sfreq=100))
f.close()

RP_training_set = torch.utils.data.ConcatDataset(datasets_list)

params = {'batch_size': 256,
          'shuffle': True,
          'num_workers': 6}
RP_training_generator = torch.utils.data.DataLoader(RP_training_set, **params)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0



## Set up downstream dataloader
datasets_list=[]
print('Loading Data')
f=open(os.path.join("..","training_names.txt"),'r')
lines = f.readlines()
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    datasets_list.append(Downstream_Dataset(path=data_file))
    d = Downstream_Dataset(path=data_file)
    print(d.labels.shape)
f.close()


dataset = torch.utils.data.ConcatDataset(datasets_list)
data_len = len(dataset)
print("dataset len is", len(dataset))

train_len = int(data_len*0.6)
val_len = data_len - train_len

training_set, validation_set = torch.utils.data.random_split(dataset, [train_len, val_len])

Loading Data
Processing tr03-0078
(973, 3000, 2)
(2000,)
Processing tr03-0079
(893, 3000, 2)
(2000,)
Processing tr05-0193
(848, 3000, 2)
fixing incorrect tpos label
fixing incorrect tpos label
(2000,)
Processing tr04-1032
(914, 3000, 2)
(2000,)
Processing tr08-0137
(979, 3000, 2)
(2000,)
Processing tr10-0740
(949, 3000, 2)
(2000,)
Processing tr05-1689
(902, 3000, 2)
fixing incorrect tpos label
fixing incorrect tpos label
fixing incorrect tpos label
fixing incorrect tpos label
fixing incorrect tpos label
fixing incorrect tpos label
(2000,)
Processing tr13-0508
(862, 3000, 2)
(2000,)
Processing tr03-0287
(864, 3000, 2)
fixing incorrect tpos label
fixing incorrect tpos label
fixing incorrect tpos label
(2000,)
Processing tr04-0649
(985, 3000, 2)
(2000,)
Processing tr07-0401
(929, 3000, 2)
(2000,)
Processing tr10-0368
(836, 3000, 2)
(2000,)
Processing tr04-0805
(1075, 3000, 2)
fixing incorrect tpos label
(2000,)
Processing tr12-0097
(857, 3000, 2)
(2000,)
Processing tr09-0478
(786, 3000, 2

In [5]:
smallest_class = smallest_class_len(training_set)
num_samples=[]
temp=1
while temp < smallest_class:
    num_samples.append(temp)
    temp*=10
num_samples.append(None)
print(num_samples)

[1, 10, 100, 1000, None]


In [6]:
def train_different_classes(RP_training_generator, tset, vset, epochs, sample_list):
    outList=[]
    for i in sample_list:
        outList.append(train_end_to_end_RP_combined(RP_training_generator, tset, vset, i, epochs, False))
        print(outList)
    return outList

In [7]:
num_samples=[1000, None]
RP_2loss_vals = train_different_classes(RP_training_generator, training_set, validation_set, 100, num_samples)

downstream batches: 20
pretext batches: 250
Start Training Full
[0.6312960554833117]
downstream batches: 68
pretext batches: 250
Start Training Full
[0.6312960554833117, 0.693194625054183]


In [8]:
print(RP_vals)

[0.3046380580840919, 0.4250541829215431, 0.5237104464672735, 0.59913307325531, 0.6680537494581708]
